# Pynq-CV-OV5640 - Application Notebook

##### This reference design illustrates how to run a imgae processing IPs and implementate OV5640 camera interface on the **Programmable Logic (PL)** using Jupyter Notebooks and Python.
<img src="./images/SystemDiagram.png" width="800" height="650"/>

##### This PYNQ Overlay contains the below accelerated image processing algorithms.
- rgb2hsv	
- subsample
- equalizehist
- gaussianBlur
- sobel
- canny
- dilation
- erosion

##### The OV5640 video streaming, imgae processing IPs and HDMI interface are connected to PL DDR using VDMA and AXI4-stream-interconnect IPs, you can choose which image processing algorithm is enabled without downloading a new bitstream. In the example application notebook, you can configure the OV5640 camera and get the captured picture and display it on the notebook, configure the image processing IPs. The processed video streaming is displayed on the HDMI monitor which connected to the PL side.

     
## Image processing IPs interconnection
The image processing IPs are connected to VDMA using AXI4-stream-interconnect IP to transceive video streaming.
<img src="./images/ImageProcessing.PNG" width="800" height="650"/>


## Import libraries

In [7]:
# 单元格1：导入库（需要修正导入路径）
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/Pynq-CV-OV5640-master')

from pynq import Overlay
from pynq.lib.iic import *
from ov5640.ov5640_config import ov5640_config  # 修正导入路径
from pynq.lib.video import *
from time import sleep
from ov5640 import dma  # 修正导入路径
import os
import shutil

## Instantiate Overlay and download bitstream

In [8]:
cv_ov5640 = Overlay("/home/xilinx/jupyter_notebooks/Pynq-CV-OV5640-master/boards/Pynq-Z2/ov5640/bitstream/cv_ov5640.bit")

/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/overlay.py:75: UserWarning: IP axi_iic_0 is of type xilinx.com:ip:axi_iic:2.0 and driver found for [xilinx.com:ip:axi_iic:2.1]. Use ignore_version=True to use this driver.
  warnings.warn(message, UserWarning)


## Instantiate I2C and configure OV5640 Camera

In [9]:
iic = AxiIIC(cv_ov5640.ip_dict['axi_iic_0'])
address = 0x3c
length = 3

In [10]:
for config in ov5640_config:
    tmp1 = config[0] >> 8;
    tmp2 = config[0] & 0xff;
    iic.send(address, bytes([tmp1, tmp2, config[1]]), length)

# Check the attributes and methods in Overlay

In [11]:
#dir(cv_ov5640.image_processing)

## Configure image processing IPs

In [12]:
bayer2rgb = cv_ov5640.v_demosaic_0
bayer2rgb.write(0x10, 1280)
bayer2rgb.write(0x18, 720)
bayer2rgb.write(0x28, 1)
bayer2rgb.write(0x00, 0x81)

rgb2hsv = cv_ov5640.image_processing.rgb2hsv_0
rgb2hsv.write(0x10, 720)
rgb2hsv.write(0x18, 1280)
rgb2hsv.write(0x20, 0)
rgb2hsv.write(0x00, 0x81)

subsample = cv_ov5640.image_processing.subsample_0
subsample.write(0x10, 720)
subsample.write(0x18, 1280)
subsample.write(0x20, 360)
subsample.write(0x28, 640)
subsample.write(0x00, 0x81)

equalizehist = cv_ov5640.image_processing.equalizeHist_0
equalizehist.write(0x10, 720)
equalizehist.write(0x18, 1280)
equalizehist.write(0x20, 0)
equalizehist.write(0x00, 0x81)

gaussianBlur = cv_ov5640.image_processing.gaussian_0
gaussianBlur.write(0x10, 720)
gaussianBlur.write(0x18, 1280)
gaussianBlur.write(0x20, 0)
gaussianBlur.write(0x00, 0x81)

sobel = cv_ov5640.image_processing.sobelFilter_0
sobel.write(0x10, 720)
sobel.write(0x18, 1280)
sobel.write(0x20, 0)
sobel.write(0x00, 0x81)

canny = cv_ov5640.image_processing.canny_edge_0
canny.write(0x10, 720)
canny.write(0x18, 1280)
canny.write(0x20, 80)
canny.write(0x28, 80)
canny.write(0x00, 0x81)

dilation = cv_ov5640.image_processing.xf_dilation_accel_0
dilation.write(0x10, 720)
dilation.write(0x18, 1280)
dilation.write(0x20, 0)
dilation.write(0x00, 0x81)

erosion = cv_ov5640.image_processing.xf_erosion_accel_0
erosion.write(0x10, 720)
erosion.write(0x18, 1280)
erosion.write(0x20, 0)
erosion.write(0x00, 0x81)

switch0 = cv_ov5640.image_processing.axis_interconnect_0.xbar
switch1 = cv_ov5640.image_processing.axis_interconnect_1.xbar

# Define a function using to choose whtich image processing IP is enabled

In [13]:
def switch_stream(switch0, switch1, Index):
    MaxIndex = 5
    switch0.write(0x00, ~0x02)
    for i in range(0,MaxIndex + 1):
        switch0.write(0x40 + 4*Index, 0x80000000)
    switch0.write(0x40 + 4*Index, 0)
    switch0.write(0x00, 0x02)

    switch1.write(0x00, ~0x02)
    for i in range(0,MaxIndex + 1):
        switch0.write(0x40 + 4*Index, 0x80000000)
    switch1.write(0x40 + 4*0, Index)
    switch1.write(0x00, 0x02)
    
switch_stream(switch0, switch1, 0)

## Configure VDMAs

In [14]:
vdma0 = cv_ov5640.axi_vdma_0
vdma1 = cv_ov5640.axi_vdma_1

## Start VDMAs，HDMI monitor will display video streaming from OV5640 camera

In [15]:
vdma0.writechannel.mode = VideoMode(1280, 720, 24)
vdma1.readchannel.mode = vdma0.writechannel.mode
vdma0.readchannel.mode = vdma0.writechannel.mode
vdma1.writechannel.mode = vdma0.writechannel.mode
vdma0.readchannel.tie(vdma0.writechannel)
vdma1.readchannel.tie(vdma1.writechannel)
vdma0.writechannel.start()
vdma0.readchannel.start()
vdma1.writechannel.start()
vdma1.readchannel.start()

## Capture and display one frame of picture, it may takes 3~5 seconds

In [16]:
while True:
    frame = vdma0.readchannel.readframe()
    if(frame.max() != 0):
        break;

In [17]:
from PIL import Image
import IPython
image = Image.fromarray(frame)
g, b, r = image.split()
image = Image.merge('RGB', [r,g,b])
IPython.display.clear_output(1)
display(image)

## Enable different image processing IPs on PL

### Enable rgb2hsv IP

In [12]:
switch_stream(switch0, switch1, 1)

### Enable subsample IP

In [13]:
switch_stream(switch0, switch1, 2)
for i in range(len(vdma1.writechannel._frames)):
    vdma1.readchannel._frames[i].fill(0)
    vdma1.readchannel._frames[i].fill(0)

### Enable equalizehist IP

In [14]:
switch_stream(switch0, switch1, 3)

### Enable gaussianBlur IP

In [15]:
switch_stream(switch0, switch1, 4)

### Enable sobel x IP

In [44]:
switch_stream(switch0, switch1, 5)

### Enable sobel y IP

In [45]:
sobel.write(0x20, 1)

### Enable canny IP

In [46]:
switch_stream(switch0, switch1, 6)

### Enable dilation IP

In [19]:
switch_stream(switch0, switch1, 7)

### Enable erosion IP

In [20]:
switch_stream(switch0, switch1, 8)

### Disable all image processing IPs

In [21]:
switch_stream(switch0, switch1, 0)

## Close VDMAs

In [22]:
vdma0.writechannel.stop()
vdma0.readchannel.stop()
vdma1.writechannel.stop()
vdma1.readchannel.stop()

In [21]:
# --- 在原有notebook末尾添加连续抓拍单元格 ---

# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image
import IPython
import matplotlib.pyplot as plt

class ContinuousCapture:
    def __init__(self, save_dir="captured_images", mirror=True, enable_rotations=True):
        self.save_dir = save_dir
        self.mirror = mirror  # 是否启用镜像
        self.enable_rotations = enable_rotations  # 是否启用8种旋转效果
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        """获取有效帧（使用您的方法）"""
        while True:
            frame = vdma0.readchannel.readframe()  # 使用vdma0
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        """颜色修正（使用您的方法）"""
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def apply_mirror(self, frame):
        """应用镜像处理"""
        if self.mirror:
            # 水平镜像（左右翻转）
            return cv2.flip(frame, 1)
        else:
            return frame
    
    def generate_all_rotations(self, frame):
        """生成8种旋转效果：原始和镜像各4个角度"""
        # 颜色修正
        corrected_frame = self.correct_color(frame)
        
        # 定义旋转角度
        angles = [0, 90, 180, 270]
        
        results = {}
        
        # 原始图像的4种旋转
        for angle in angles:
            if angle == 0:
                rotated = corrected_frame
            else:
                # 使用PIL进行高质量旋转
                pil_img = Image.fromarray(corrected_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"original_{angle}"] = rotated
        
        # 镜像后的4种旋转
        mirrored_frame = cv2.flip(corrected_frame, 1)  # 水平镜像
        for angle in angles:
            if angle == 0:
                rotated = mirrored_frame
            else:
                pil_img = Image.fromarray(mirrored_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"mirrored_{angle}"] = rotated
        
        return results
        
    def save_frame(self, frame, prefix="frame"):
        """保存帧到文件"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        
        if self.enable_rotations:
            # 保存8种旋转效果
            return self.save_all_rotations(frame, prefix, timestamp)
        else:
            # 保存单张图片（原有功能）
            filename = f"{self.save_dir}/{prefix}_{timestamp}_{self.frame_count:06d}.jpg"
            
            # 使用您的颜色修正方法
            corrected_frame = self.correct_color(frame)
            
            # 应用镜像处理
            mirrored_frame = self.apply_mirror(corrected_frame)
            
            # 转换颜色空间 RGB to BGR for OpenCV保存
            frame_bgr = cv2.cvtColor(mirrored_frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(filename, frame_bgr)
            
            self.frame_count += 1
            return [filename]
    
    def save_all_rotations(self, frame, prefix, timestamp):
        """保存8种旋转效果"""
        saved_files = []
        
        # 生成所有旋转效果
        rotations = self.generate_all_rotations(frame)
        
        for rotation_name, rotated_frame in rotations.items():
            filename = f"{self.save_dir}/{prefix}_{timestamp}_{rotation_name}_{self.frame_count:06d}.jpg"
            
            # 转换颜色空间 RGB to BGR for OpenCV保存
            frame_bgr = cv2.cvtColor(rotated_frame, cv2.COLOR_RGB2BGR)
            success = cv2.imwrite(filename, frame_bgr)
            
            if success:
                saved_files.append((rotation_name, filename))
        
        self.frame_count += 1
        return saved_files
        
    def capture_continuous(self, duration=30, interval=1.0):
        """连续抓拍
        
        Args:
            duration: 总抓拍时间（秒）
            interval: 抓拍间隔（秒）
        """
        mirror_status = "开启" if self.mirror else "关闭"
        rotation_status = "开启(8张/组)" if self.enable_rotations else "关闭(1张/组)"
        
        print(f"开始连续抓拍: 持续时间{duration}秒, 间隔{interval}秒")
        print(f"镜像功能: {mirror_status}")
        print(f"旋转效果: {rotation_status}")
        print(f"图像将保存到: {self.save_dir}")
        
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                # 检查是否到达抓拍间隔
                if current_time - last_capture_time >= interval:
                    # 使用您的方法获取有效帧
                    frame = self.get_valid_frame()
                    
                    # 保存帧
                    saved_files = self.save_frame(frame)
                    elapsed = current_time - start_time
                    
                    if self.enable_rotations:
                        print(f"[{elapsed:6.1f}s] 第{self.frame_count}组 - 生成8张:")
                        for rotation_name, filename in saved_files:
                            print(f"       {rotation_name}: {os.path.basename(filename)}")
                    else:
                        filename = saved_files[0]
                        print(f"[{elapsed:6.1f}s] 保存: {os.path.basename(filename)} (第{self.frame_count}帧)")
                    
                    last_capture_time = current_time
                
                # 短暂休眠以减少CPU占用
                time.sleep(0.01)
                
            total_images = self.frame_count * (8 if self.enable_rotations else 1)
            print(f"\n抓拍完成! 共保存 {total_images} 张图像 ({self.frame_count}组)")
            
        except KeyboardInterrupt:
            total_images = self.frame_count * (8 if self.enable_rotations else 1)
            print(f"\n用户中断! 已保存 {total_images} 张图像 ({self.frame_count}组)")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

# 创建抓拍实例（默认开启镜像和旋转效果）
capture = ContinuousCapture(mirror=True, enable_rotations=True)

# --- 单元格：快速测试连续抓拍 ---
def quick_capture_test():
    """快速测试连续抓拍功能"""
    print("=== 快速抓拍测试 ===")
    # 抓拍5秒，每秒1组（8张）
    capture.capture_continuous(duration=5, interval=1.0)

# --- 单元格：自定义抓拍参数 ---
def custom_capture():
    """自定义抓拍参数"""
    duration = 1    # 抓拍总时间（秒）
    interval = 2   # 抓拍间隔（秒）
    
    print(f"开始自定义抓拍: {duration}秒, 每{interval}秒一组")
    capture.capture_continuous(duration=duration, interval=interval)

# --- 单元格：实时预览+抓拍 ---
def preview_and_capture(capture_interval=3.0, preview_duration=30):
    """实时预览并定时抓拍"""
    %matplotlib inline
    
    mirror_status = "开启" if capture.mirror else "关闭"
    rotation_status = "开启(8张/组)" if capture.enable_rotations else "关闭(1张/组)"
    
    print(f"实时预览模式: {preview_duration}秒, 每{capture_interval}秒抓拍一次")
    print(f"镜像功能: {mirror_status}")
    print(f"旋转效果: {rotation_status}")
    
    plt.ion()
    fig, ax = plt.subplots(figsize=(10, 6))
    
    start_time = time.time()
    last_capture_time = start_time
    capture_count = 0
    
    try:
        while time.time() - start_time < preview_duration:
            # 使用您的方法获取有效帧
            frame = capture.get_valid_frame()
            current_time = time.time()
            
            # 检查是否需要抓拍
            if current_time - last_capture_time >= capture_interval:
                saved_files = capture.save_frame(frame, prefix="preview_capture")
                capture_count += 1
                
                if capture.enable_rotations:
                    print(f"抓拍第{capture_count}组 - 8张图片")
                else:
                    print(f"抓拍: {saved_files[0]}")
                    
                last_capture_time = current_time
            
            # 更新预览（使用修正后的颜色+镜像）
            corrected_frame = capture.correct_color(frame)
            mirrored_frame = capture.apply_mirror(corrected_frame)
            
            ax.clear()
            ax.imshow(mirrored_frame)
            images_per_group = 8 if capture.enable_rotations else 1
            total_images = capture_count * images_per_group
            ax.set_title(f'实时预览 - 已抓拍 {total_images} 张 - 剩余时间: {preview_duration - (current_time - start_time):.1f}秒')
            ax.axis('off')
            
            plt.tight_layout()
            plt.pause(0.1)  # 控制预览帧率
            
    except KeyboardInterrupt:
        print("用户停止预览")
    finally:
        plt.ioff()
        plt.close()
        images_per_group = 8 if capture.enable_rotations else 1
        total_images = capture_count * images_per_group
        print(f"预览结束，共抓拍 {total_images} 张图像")

# --- 单元格：测试8种旋转效果 ---
def test_all_rotations():
    """测试8种旋转效果"""
    print("=== 测试8种旋转效果 ===")
    
    # 使用您的方法获取和显示一帧
    while True:
        frame = vdma0.readchannel.readframe()
        if frame is not None and frame.max() != 0:
            break
    
    # 生成8种旋转效果
    rotations = capture.generate_all_rotations(frame)
    
    # 显示所有效果
    IPython.display.clear_output(wait=True)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    # 显示顺序：原始0°, 90°, 180°, 270°, 镜像0°, 90°, 180°, 270°
    display_order = [
        'original_0', 'original_90', 'original_180', 'original_270',
        'mirrored_0', 'mirrored_90', 'mirrored_180', 'mirrored_270'
    ]
    
    titles = {
        'original_0': '原始 0°',
        'original_90': '原始 90°',
        'original_180': '原始 180°',
        'original_270': '原始 270°',
        'mirrored_0': '镜像 0°',
        'mirrored_90': '镜像 90°',
        'mirrored_180': '镜像 180°',
        'mirrored_270': '镜像 270°'
    }
    
    for i, rotation_name in enumerate(display_order):
        if rotation_name in rotations:
            axes[i].imshow(rotations[rotation_name])
            axes[i].set_title(titles[rotation_name])
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 8种旋转效果测试完成")
    return rotations

# --- 单元格：功能控制 ---
def toggle_mirror(enable=True):
    """切换镜像功能"""
    capture.mirror = enable
    status = "开启" if enable else "关闭"
    print(f"镜像功能已{status}")

def toggle_rotations(enable=True):
    """切换旋转效果功能"""
    capture.enable_rotations = enable
    status = "开启(8张/组)" if enable else "关闭(1张/组)"
    print(f"旋转效果功能已{status}")

# --- 使用说明单元格 ---
print("""
=== 增强版连续抓拍系统 ===

可用函数:
1. quick_capture_test()    - 快速测试(5秒, 每秒1组)
2. custom_capture()        - 自定义抓拍
3. preview_and_capture()   - 实时预览+抓拍
4. test_all_rotations()    - 测试8种旋转效果
5. toggle_mirror(True/False)   - 开启/关闭镜像
6. toggle_rotations(True/False) - 开启/关闭旋转效果

功能说明:
- 镜像功能: 控制是否对图片进行水平镜像
- 旋转效果: 控制是否生成8种旋转效果

8种旋转效果:
- 原始图像: 0°, 90°, 180°, 270°
- 镜像图像: 0°, 90°, 180°, 270°

文件命名格式:
开启旋转效果时:
  frame_时间戳_效果名_序号.jpg
  效果名: original_0, original_90, original_180, original_270
          mirrored_0, mirrored_90, mirrored_180, mirrored_270

关闭旋转效果时:
  frame_时间戳_序号.jpg

当前状态:
- 镜像: 开启
- 旋转效果: 开启(8张/组)

使用步骤:
1. 先运行 test_all_rotations() 查看效果
2. 使用 toggle_xxx() 调整功能设置
3. 选择其他函数执行抓拍
4. 查看captured_images文件夹中的结果
""")

# 自动创建保存目录
os.makedirs("captured_images", exist_ok=True)
print("增强版抓拍系统就绪!")


=== 增强版连续抓拍系统 ===

可用函数:
1. quick_capture_test()    - 快速测试(5秒, 每秒1组)
2. custom_capture()        - 自定义抓拍
3. preview_and_capture()   - 实时预览+抓拍
4. test_all_rotations()    - 测试8种旋转效果
5. toggle_mirror(True/False)   - 开启/关闭镜像
6. toggle_rotations(True/False) - 开启/关闭旋转效果

功能说明:
- 镜像功能: 控制是否对图片进行水平镜像
- 旋转效果: 控制是否生成8种旋转效果

8种旋转效果:
- 原始图像: 0°, 90°, 180°, 270°
- 镜像图像: 0°, 90°, 180°, 270°

文件命名格式:
开启旋转效果时:
  frame_时间戳_效果名_序号.jpg
  效果名: original_0, original_90, original_180, original_270
          mirrored_0, mirrored_90, mirrored_180, mirrored_270

关闭旋转效果时:
  frame_时间戳_序号.jpg

当前状态:
- 镜像: 开启
- 旋转效果: 开启(8张/组)

使用步骤:
1. 先运行 test_all_rotations() 查看效果
2. 使用 toggle_xxx() 调整功能设置
3. 选择其他函数执行抓拍
4. 查看captured_images文件夹中的结果

增强版抓拍系统就绪!


In [22]:
# 测试8种旋转效果
test_all_rotations()

# 开启/关闭功能
toggle_rotations(True)   # 开启8种旋转

# 开始抓拍
custom_capture()  

/tmp/ipykernel_844/1970310862.py:296: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_844/1970310862.py:296: UserWarning: Glyph 22987 (\N{CJK UNIFIED IDEOGRAPH-59CB}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_844/1970310862.py:296: UserWarning: Glyph 38236 (\N{CJK UNIFIED IDEOGRAPH-955C}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_844/1970310862.py:296: UserWarning: Glyph 20687 (\N{CJK UNIFIED IDEOGRAPH-50CF}) missing from current font.
  plt.tight_layout()


✅ 8种旋转效果测试完成
旋转效果功能已开启(8张/组)
开始自定义抓拍: 1秒, 每2秒一组
开始连续抓拍: 持续时间1秒, 间隔2秒
镜像功能: 开启
旋转效果: 开启(8张/组)
图像将保存到: captured_images

抓拍完成! 共保存 0 张图像 (0组)


In [23]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        """获取有效帧"""
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        """颜色修正"""
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def generate_all_rotations(self, frame):
        """生成8种旋转效果：原始和镜像各4个角度"""
        # 颜色修正
        corrected_frame = self.correct_color(frame)
        
        # 定义旋转角度
        angles = [0, 90, 180, 270]
        
        results = {}
        
        # 原始图像的4种旋转
        for angle in angles:
            if angle == 0:
                rotated = corrected_frame
            else:
                # 使用PIL进行高质量旋转
                pil_img = Image.fromarray(corrected_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"original_{angle}"] = rotated
        
        # 镜像后的4种旋转
        mirrored_frame = cv2.flip(corrected_frame, 1)  # 水平镜像
        for angle in angles:
            if angle == 0:
                rotated = mirrored_frame
            else:
                pil_img = Image.fromarray(mirrored_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"mirrored_{angle}"] = rotated
        
        return results
        
    def save_all_rotations(self, frame, prefix="frame", timestamp=None):
        """保存8种旋转效果"""
        if timestamp is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            
        saved_files = []
        
        # 生成所有旋转效果
        rotations = self.generate_all_rotations(frame)
        
        for rotation_name, rotated_frame in rotations.items():
            filename = f"{self.save_dir}/{prefix}_{timestamp}_{rotation_name}_{self.frame_count:06d}.jpg"
            
            # 转换颜色空间 RGB to BGR for OpenCV保存
            frame_bgr = cv2.cvtColor(rotated_frame, cv2.COLOR_RGB2BGR)
            success = cv2.imwrite(filename, frame_bgr)
            
            if success:
                saved_files.append((rotation_name, filename))
        
        self.frame_count += 1
        return saved_files
        
    def capture_continuous(self, duration=30, interval=1.0):
        """连续抓拍（生成8种旋转效果）
        
        Args:
            duration: 总抓拍时间（秒）
            interval: 抓拍间隔（秒）
        """
        print(f"开始连续抓拍")
        print(f"持续时间: {duration}秒")
        print(f"抓拍间隔: {interval}秒")
        print(f"每张照片生成8种旋转效果")
        print(f"图像保存到: {self.save_dir}")
        
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                # 检查是否到达抓拍间隔
                if current_time - last_capture_time >= interval:
                    # 获取有效帧
                    frame = self.get_valid_frame()
                    
                    # 保存8种旋转效果
                    saved_files = self.save_all_rotations(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 第{self.frame_count}组 - 生成8张:")
                    for rotation_name, filename in saved_files:
                        print(f"       {rotation_name}: {os.path.basename(filename)}")
                    
                    last_capture_time = current_time
                
                # 短暂休眠以减少CPU占用
                time.sleep(0.01)
                
            total_images = self.frame_count * 8
            print(f"\n抓拍完成! 共生成 {total_images} 张图像 ({self.frame_count}组×8张)")
            
        except KeyboardInterrupt:
            total_images = self.frame_count * 8
            print(f"\n用户中断! 共生成 {total_images} 张图像 ({self.frame_count}组×8张)")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

# 创建抓拍实例
capture = ContinuousCapture()

# --- 单元格：自定义抓拍参数 ---
def custom_capture(duration=10, interval=1.0):
    """自定义抓拍参数
    
    Args:
        duration: 抓拍总时间（秒）
        interval: 抓拍间隔（秒）
    """
    print(f"开始自定义抓拍")
    print(f"持续时间: {duration}秒")
    print(f"抓拍间隔: {interval}秒")
    print(f"每{interval}秒拍摄一组8张旋转图片")
    
    capture.capture_continuous(duration=duration, interval=interval)

# --- 使用说明单元格 ---
print("""
=== 八种旋转效果抓拍系统 ===

唯一功能:
custom_capture(duration, interval) - 自定义抓拍参数

参数说明:
- duration: 总抓拍时间(秒)
- interval: 抓拍间隔(秒)

旋转效果说明:
每张照片自动生成8种效果:
- 原始图像: 0°, 90°, 180°, 270°
- 镜像图像: 0°, 90°, 180°, 270°

文件命名格式:
frame_时间戳_效果名_序号.jpg
效果名: 
  original_0, original_90, original_180, original_270
  mirrored_0, mirrored_90, mirrored_180, mirrored_270

使用示例:
custom_capture(duration=10, interval=1.0)  # 10秒, 每秒1组8张
custom_capture(duration=30, interval=2.0)  # 30秒, 每2秒1组8张

图像保存位置: captured_images/
""")

# 自动创建保存目录
os.makedirs("captured_images", exist_ok=True)
print("抓拍系统就绪!")


=== 八种旋转效果抓拍系统 ===

唯一功能:
custom_capture(duration, interval) - 自定义抓拍参数

参数说明:
- duration: 总抓拍时间(秒)
- interval: 抓拍间隔(秒)

旋转效果说明:
每张照片自动生成8种效果:
- 原始图像: 0°, 90°, 180°, 270°
- 镜像图像: 0°, 90°, 180°, 270°

文件命名格式:
frame_时间戳_效果名_序号.jpg
效果名: 
  original_0, original_90, original_180, original_270
  mirrored_0, mirrored_90, mirrored_180, mirrored_270

使用示例:
custom_capture(duration=10, interval=1.0)  # 10秒, 每秒1组8张
custom_capture(duration=30, interval=2.0)  # 30秒, 每2秒1组8张

图像保存位置: captured_images/

抓拍系统就绪!


In [26]:
# 抓拍10秒，每秒1组8张
custom_capture(duration=2, interval=1.0)


开始自定义抓拍
持续时间: 2秒
抓拍间隔: 1.0秒
每1.0秒拍摄一组8张旋转图片
开始连续抓拍
持续时间: 2秒
抓拍间隔: 1.0秒
每张照片生成8种旋转效果
图像保存到: captured_images
[   1.0s] 第2组 - 生成8张:
       original_0: frame_20250928_161416_730799_original_0_000001.jpg
       original_90: frame_20250928_161416_730799_original_90_000001.jpg
       original_180: frame_20250928_161416_730799_original_180_000001.jpg
       original_270: frame_20250928_161416_730799_original_270_000001.jpg
       mirrored_0: frame_20250928_161416_730799_mirrored_0_000001.jpg
       mirrored_90: frame_20250928_161416_730799_mirrored_90_000001.jpg
       mirrored_180: frame_20250928_161416_730799_mirrored_180_000001.jpg
       mirrored_270: frame_20250928_161416_730799_mirrored_270_000001.jpg

抓拍完成! 共生成 16 张图像 (2组×8张)


In [27]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def generate_all_rotations(self, frame):
        corrected_frame = self.correct_color(frame)
        angles = [0, 90, 180, 270]
        results = {}
        
        for angle in angles:
            if angle == 0:
                rotated = corrected_frame
            else:
                pil_img = Image.fromarray(corrected_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"original_{angle}"] = rotated
        
        mirrored_frame = cv2.flip(corrected_frame, 1)
        for angle in angles:
            if angle == 0:
                rotated = mirrored_frame
            else:
                pil_img = Image.fromarray(mirrored_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"mirrored_{angle}"] = rotated
        
        return results
        
    def save_all_rotations(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        saved_files = []
        rotations = self.generate_all_rotations(frame)
        
        for rotation_name, rotated_frame in rotations.items():
            filename = f"{self.save_dir}/frame_{timestamp}_{rotation_name}_{self.frame_count:06d}.jpg"
            frame_bgr = cv2.cvtColor(rotated_frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(filename, frame_bgr)
            saved_files.append((rotation_name, filename))
        
        self.frame_count += 1
        return saved_files
        
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    saved_files = self.save_all_rotations(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 第{self.frame_count}组 - 生成8张:")
                    for rotation_name, filename in saved_files:
                        print(f"       {rotation_name}: {os.path.basename(filename)}")
                    
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            total_images = self.frame_count * 8
            print(f"\n抓拍完成! 共生成 {total_images} 张图像")
            
        except KeyboardInterrupt:
            total_images = self.frame_count * 8
            print(f"\n用户中断! 共生成 {total_images} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)


In [29]:
custom_capture(duration=2, interval=1.0)

[   1.0s] 第2组 - 生成8张:
       original_0: frame_20250928_162113_476890_original_0_000001.jpg
       original_90: frame_20250928_162113_476890_original_90_000001.jpg
       original_180: frame_20250928_162113_476890_original_180_000001.jpg
       original_270: frame_20250928_162113_476890_original_270_000001.jpg
       mirrored_0: frame_20250928_162113_476890_mirrored_0_000001.jpg
       mirrored_90: frame_20250928_162113_476890_mirrored_90_000001.jpg
       mirrored_180: frame_20250928_162113_476890_mirrored_180_000001.jpg
       mirrored_270: frame_20250928_162113_476890_mirrored_270_000001.jpg

抓拍完成! 共生成 16 张图像


In [32]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def save_frame(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = f"{self.save_dir}/frame_{timestamp}_{self.frame_count:06d}.jpg"
        
        corrected_frame = self.correct_color(frame)
        mirrored_frame = cv2.flip(corrected_frame, 1)
        frame_bgr = cv2.cvtColor(mirrored_frame, cv2.COLOR_RGB2BGR)
        cv2.imwrite(filename, frame_bgr)
        
        self.frame_count += 1
        return filename
    
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    filename = self.save_frame(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 保存: {os.path.basename(filename)} (第{self.frame_count}帧)")
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            print(f"\n抓拍完成! 共保存 {self.frame_count} 张图像")
            
        except KeyboardInterrupt:
            print(f"\n用户中断! 已保存 {self.frame_count} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)

In [33]:
custom_capture(duration=4, interval=1.5)

[   1.5s] 保存: frame_20250928_163355_977263_000000.jpg (第1帧)
[   3.0s] 保存: frame_20250928_163357_493978_000001.jpg (第2帧)

抓拍完成! 共保存 2 张图像


In [13]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def save_frame(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = f"{self.save_dir}/frame_{timestamp}_{self.frame_count:06d}.jpg"
        
        corrected_frame = self.correct_color(frame)
        mirrored_frame = cv2.flip(corrected_frame, 1)
        
        # 显示原始尺寸
        original_height, original_width = frame.shape[:2]
        print(f"原始尺寸: {original_width}×{original_height}")
        
        # 放大两倍
        height, width = mirrored_frame.shape[:2]
        enlarged_frame = cv2.resize(mirrored_frame, (width*2, height*2), interpolation=cv2.INTER_LINEAR)
        
        # 显示放大后尺寸
        enlarged_height, enlarged_width = enlarged_frame.shape[:2]
        print(f"放大后尺寸: {enlarged_width}×{enlarged_height}")
        
        frame_bgr = cv2.cvtColor(enlarged_frame, cv2.COLOR_RGB2BGR)
        success = cv2.imwrite(filename, frame_bgr)
        
        if success:
            file_size = os.path.getsize(filename)
            print(f"文件大小: {file_size} bytes")
        
        self.frame_count += 1
        return filename
    
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    filename = self.save_frame(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 保存: {os.path.basename(filename)} (第{self.frame_count}帧)")
                    print("-" * 50)
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            print(f"\n抓拍完成! 共保存 {self.frame_count} 张图像")
            
        except KeyboardInterrupt:
            print(f"\n用户中断! 已保存 {self.frame_count} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)

custom_capture(duration=2, interval=1.5)

原始尺寸: 1280×720
放大后尺寸: 2560×1440
文件大小: 551649 bytes
[   1.5s] 保存: frame_20250928_182320_674181_000000.jpg (第1帧)
--------------------------------------------------

抓拍完成! 共保存 1 张图像


In [32]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def save_frame(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = f"{self.save_dir}/frame_{timestamp}_{self.frame_count:06d}.jpg"
        
        corrected_frame = self.correct_color(frame)
        mirrored_frame = cv2.flip(corrected_frame, 1)
        
        # 显示原始尺寸
        original_height, original_width = mirrored_frame.shape[:2]
        print(f"原始尺寸: {original_width}×{original_height}")
        
        # 计算裁剪区域（中心区域，大小为原图的2/3，相当于1.5倍放大）
        crop_width = int(original_width / 1.2)
        crop_height = int(original_height / 1.)
        start_x = (original_width - crop_width) // 2
        start_y = (original_height - crop_height) // 2
        
        # 裁剪中心区域
        cropped_frame = mirrored_frame[start_y:start_y+crop_height, start_x:start_x+crop_width]
        
        # 放大到1280×720
        enlarged_frame = cv2.resize(cropped_frame, (1280, 720), interpolation=cv2.INTER_LINEAR)
        
        # 裁剪为720×720正方形（居中裁剪）
        final_height, final_width = enlarged_frame.shape[:2]
        square_size = 720
        start_x_square = (final_width - square_size) // 2
        start_y_square = (final_height - square_size) // 2
        square_frame = enlarged_frame[start_y_square:start_y_square+square_size, start_x_square:start_x_square+square_size]
        
        print(f"裁剪后尺寸: {crop_width}×{crop_height}")
        print(f"放大后尺寸: 1280×720")
        print(f"最终尺寸: 720×720")
        
        frame_bgr = cv2.cvtColor(square_frame, cv2.COLOR_RGB2BGR)
        success = cv2.imwrite(filename, frame_bgr)
        
        if success:
            file_size = os.path.getsize(filename)
            print(f"文件大小: {file_size} bytes")
        
        self.frame_count += 1
        return filename
    
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    filename = self.save_frame(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 保存: {os.path.basename(filename)} (第{self.frame_count}帧)")
                    print("-" * 50)
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            print(f"\n抓拍完成! 共保存 {self.frame_count} 张图像")
            
        except KeyboardInterrupt:
            print(f"\n用户中断! 已保存 {self.frame_count} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)

custom_capture(duration=2, interval=1.5)

原始尺寸: 1280×720
裁剪后尺寸: 1066×720
放大后尺寸: 1280×720
最终尺寸: 720×720
文件大小: 16042 bytes
[   1.5s] 保存: frame_20250928_190522_322395_000000.jpg (第1帧)
--------------------------------------------------

抓拍完成! 共保存 1 张图像


In [38]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def save_frame(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = f"{self.save_dir}/frame_{timestamp}_{self.frame_count:06d}.jpg"
        
        corrected_frame = self.correct_color(frame)
        mirrored_frame = cv2.flip(corrected_frame, 1)
        
        # 显示原始尺寸
        original_height, original_width = mirrored_frame.shape[:2]
        print(f"原始尺寸: {original_width}×{original_height}")
        
        # 计算裁剪区域（中心区域，大小为原图的1/1.2）
        crop_width = int(original_width / 1.2)
        crop_height = int(original_height / 1.2)
        start_x = (original_width - crop_width) // 2
        start_y = (original_height - crop_height) // 2
        
        # 裁剪中心区域
        cropped_frame = mirrored_frame[start_y:start_y+crop_height, start_x:start_x+crop_width]
        
        print(f"裁剪后尺寸: {crop_width}×{crop_height}")
        
        # 正方形裁剪（取较小边作为正方形边长）
        square_size = min(crop_width, crop_height)
        start_x_square = (crop_width - square_size) // 2
        start_y_square = (crop_height - square_size) // 2
        square_frame = cropped_frame[start_y_square:start_y_square+square_size, start_x_square:start_x_square+square_size]
        
        print(f"最终尺寸: {square_size}×{square_size}")
        
        frame_bgr = cv2.cvtColor(square_frame, cv2.COLOR_RGB2BGR)
        success = cv2.imwrite(filename, frame_bgr)
        
        if success:
            file_size = os.path.getsize(filename)
            print(f"文件大小: {file_size} bytes")
        
        self.frame_count += 1
        return filename
    
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    filename = self.save_frame(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 保存: {os.path.basename(filename)} (第{self.frame_count}帧)")
                    print("-" * 50)
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            print(f"\n抓拍完成! 共保存 {self.frame_count} 张图像")
            
        except KeyboardInterrupt:
            print(f"\n用户中断! 已保存 {self.frame_count} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)

custom_capture(duration=2, interval=1.5)

原始尺寸: 1280×720
裁剪后尺寸: 1066×600
最终尺寸: 600×600
文件大小: 75066 bytes
[   1.5s] 保存: frame_20250928_192738_108299_000000.jpg (第1帧)
--------------------------------------------------

抓拍完成! 共保存 1 张图像


In [35]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def generate_all_rotations(self, frame):
        corrected_frame = self.correct_color(frame)
        angles = [0, 90, 180, 270]
        results = {}
        
        for angle in angles:
            if angle == 0:
                rotated = corrected_frame
            else:
                pil_img = Image.fromarray(corrected_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"original_{angle}"] = rotated
        
        mirrored_frame = cv2.flip(corrected_frame, 1)
        for angle in angles:
            if angle == 0:
                rotated = mirrored_frame
            else:
                pil_img = Image.fromarray(mirrored_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"mirrored_{angle}"] = rotated
        
        return results
        
    def save_all_rotations(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        saved_files = []
        rotations = self.generate_all_rotations(frame)
        
        for rotation_name, rotated_frame in rotations.items():
            filename = f"{self.save_dir}/frame_{timestamp}_{rotation_name}_{self.frame_count:06d}.jpg"
            frame_bgr = cv2.cvtColor(rotated_frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(filename, frame_bgr)
            saved_files.append((rotation_name, filename))
        
        self.frame_count += 1
        return saved_files
        
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    saved_files = self.save_all_rotations(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 第{self.frame_count}组 - 生成8张:")
                    for rotation_name, filename in saved_files:
                        print(f"       {rotation_name}: {os.path.basename(filename)}")
                    
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            total_images = self.frame_count * 8
            print(f"\n抓拍完成! 共生成 {total_images} 张图像")
            
        except KeyboardInterrupt:
            total_images = self.frame_count * 8
            print(f"\n用户中断! 共生成 {total_images} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)



custom_capture(duration=2, interval=1.5)

[   1.5s] 第1组 - 生成8张:
       original_0: frame_20250928_191906_598321_original_0_000000.jpg
       original_90: frame_20250928_191906_598321_original_90_000000.jpg
       original_180: frame_20250928_191906_598321_original_180_000000.jpg
       original_270: frame_20250928_191906_598321_original_270_000000.jpg
       mirrored_0: frame_20250928_191906_598321_mirrored_0_000000.jpg
       mirrored_90: frame_20250928_191906_598321_mirrored_90_000000.jpg
       mirrored_180: frame_20250928_191906_598321_mirrored_180_000000.jpg
       mirrored_270: frame_20250928_191906_598321_mirrored_270_000000.jpg

抓拍完成! 共生成 8 张图像


In [37]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def center_crop_1_2x(self, frame):
        """中心裁剪（1.2倍放大）"""
        height, width = frame.shape[:2]
        
        # 计算裁剪区域（中心区域，大小为原图的1/1.2）
        crop_width = int(width / 1.2)
        crop_height = int(height / 1.2)
        start_x = (width - crop_width) // 2
        start_y = (height - crop_height) // 2
        
        # 裁剪中心区域
        cropped_frame = frame[start_y:start_y+crop_height, start_x:start_x+crop_width]
        
        return cropped_frame
    
    def square_crop(self, frame):
        """正方形裁剪"""
        height, width = frame.shape[:2]
        
        # 取较小边作为正方形边长
        square_size = min(height, width)
        start_x = (width - square_size) // 2
        start_y = (height - square_size) // 2
        
        # 裁剪正方形区域
        square_frame = frame[start_y:start_y+square_size, start_x:start_x+square_size]
        
        return square_frame
    
    def generate_all_rotations(self, frame):
        # 先进行中心裁剪（1.2倍放大）
        cropped_frame = self.center_crop_1_2x(frame)
        
        # 再进行正方形裁剪
        square_frame = self.square_crop(cropped_frame)
        
        # 颜色修正
        corrected_frame = self.correct_color(square_frame)
        
        angles = [0, 90, 180, 270]
        results = {}
        
        for angle in angles:
            if angle == 0:
                rotated = corrected_frame
            else:
                pil_img = Image.fromarray(corrected_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"original_{angle}"] = rotated
        
        mirrored_frame = cv2.flip(corrected_frame, 1)
        for angle in angles:
            if angle == 0:
                rotated = mirrored_frame
            else:
                pil_img = Image.fromarray(mirrored_frame)
                rotated_pil = pil_img.rotate(angle, expand=True)
                rotated = np.array(rotated_pil)
            results[f"mirrored_{angle}"] = rotated
        
        return results
        
    def save_all_rotations(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        saved_files = []
        rotations = self.generate_all_rotations(frame)
        
        for rotation_name, rotated_frame in rotations.items():
            filename = f"{self.save_dir}/frame_{timestamp}_{rotation_name}_{self.frame_count:06d}.jpg"
            frame_bgr = cv2.cvtColor(rotated_frame, cv2.COLOR_RGB2BGR)
            cv2.imwrite(filename, frame_bgr)
            saved_files.append((rotation_name, filename))
        
        self.frame_count += 1
        return saved_files
        
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    saved_files = self.save_all_rotations(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 第{self.frame_count}组 - 生成8张:")
                    for rotation_name, filename in saved_files:
                        print(f"       {rotation_name}: {os.path.basename(filename)}")
                    
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            total_images = self.frame_count * 8
            print(f"\n抓拍完成! 共生成 {total_images} 张图像")
            
        except KeyboardInterrupt:
            total_images = self.frame_count * 8
            print(f"\n用户中断! 共生成 {total_images} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)
custom_capture(duration=2, interval=1.0)

[   1.0s] 第1组 - 生成8张:
       original_0: frame_20250928_192136_096751_original_0_000000.jpg
       original_90: frame_20250928_192136_096751_original_90_000000.jpg
       original_180: frame_20250928_192136_096751_original_180_000000.jpg
       original_270: frame_20250928_192136_096751_original_270_000000.jpg
       mirrored_0: frame_20250928_192136_096751_mirrored_0_000000.jpg
       mirrored_90: frame_20250928_192136_096751_mirrored_90_000000.jpg
       mirrored_180: frame_20250928_192136_096751_mirrored_180_000000.jpg
       mirrored_270: frame_20250928_192136_096751_mirrored_270_000000.jpg

抓拍完成! 共生成 8 张图像


In [31]:
# --- 单元格：连续抓拍功能 ---
import time
import os
from datetime import datetime
import cv2
import numpy as np
from PIL import Image

class ContinuousCapture:
    def __init__(self, save_dir="captured_images"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.frame_count = 0
        
    def get_valid_frame(self):
        while True:
            frame = vdma0.readchannel.readframe()
            if frame is not None and frame.max() != 0:
                return frame
    
    def correct_color(self, frame):
        image = Image.fromarray(frame)
        g, b, r = image.split()
        corrected_image = Image.merge('RGB', [r, g, b])
        return np.array(corrected_image)
    
    def save_frame(self, frame):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = f"{self.save_dir}/frame_{timestamp}_{self.frame_count:06d}.jpg"
        
        corrected_frame = self.correct_color(frame)
        mirrored_frame = cv2.flip(corrected_frame, 1)
        
        # 显示原始尺寸
        original_height, original_width = mirrored_frame.shape[:2]
        print(f"原始尺寸: {original_width}×{original_height}")
        
        # 计算裁剪区域（中心区域，大小为原图的1/1.2）
        crop_width = int(original_width / 1.2)
        crop_height = int(original_height / 1.2)
        start_x = (original_width - crop_width) // 2
        start_y = (original_height - crop_height) // 2
        
        # 裁剪中心区域
        cropped_frame = mirrored_frame[start_y:start_y+crop_height, start_x:start_x+crop_width]
        
        print(f"裁剪后尺寸: {crop_width}×{crop_height}")
        
        # 正方形裁剪（取较小边作为正方形边长）
        square_size = min(crop_width, crop_height)
        start_x_square = (crop_width - square_size) // 2
        start_y_square = (crop_height - square_size) // 2
        square_frame = cropped_frame[start_y_square:start_y_square+square_size, start_x_square:start_x_square+square_size]
        
        print(f"最终尺寸: {square_size}×{square_size}")
        
        frame_bgr = cv2.cvtColor(square_frame, cv2.COLOR_RGB2BGR)
        success = cv2.imwrite(filename, frame_bgr)
        
        if success:
            file_size = os.path.getsize(filename)
            print(f"文件大小: {file_size} bytes")
        
        self.frame_count += 1
        return filename
    
    def capture_continuous(self, duration=30, interval=1.0):
        start_time = time.time()
        last_capture_time = start_time
        
        try:
            while time.time() - start_time < duration:
                current_time = time.time()
                
                if current_time - last_capture_time >= interval:
                    frame = self.get_valid_frame()
                    filename = self.save_frame(frame)
                    elapsed = current_time - start_time
                    
                    print(f"[{elapsed:6.1f}s] 保存: {os.path.basename(filename)} (第{self.frame_count}帧)")
                    print("-" * 50)
                    last_capture_time = current_time
                
                time.sleep(0.01)
                
            print(f"\n抓拍完成! 共保存 {self.frame_count} 张图像")
            
        except KeyboardInterrupt:
            print(f"\n用户中断! 已保存 {self.frame_count} 张图像")
        except Exception as e:
            print(f"抓拍过程中出错: {e}")

capture = ContinuousCapture()

def custom_capture(duration=10, interval=1.0):
    capture.capture_continuous(duration=duration, interval=interval)

os.makedirs("captured_images", exist_ok=True)

custom_capture(duration=2, interval=1.5)

原始尺寸: 1280×720
裁剪后尺寸: 1066×600
最终尺寸: 600×600
文件大小: 70666 bytes
[   1.5s] 保存: frame_20250928_202814_171666_000000.jpg (第1帧)
--------------------------------------------------

抓拍完成! 共保存 1 张图像


In [1]:
# 检查是否有所需库
try:
    import torch
    print(f"PyTorch版本: {torch.__version__}")
except ImportError:
    print("需要安装PyTorch")

try:
    from PIL import Image
    print("PIL可用")
except ImportError:
    print("需要安装Pillow")

try:
    import torchvision
    print("TorchVision可用")
except ImportError:
    print("需要安装torchvision")

需要安装PyTorch
PIL可用
需要安装torchvision


In [2]:
import torch
print(f"PyTorch版本: {torch.__version__}")
import torchvision
print(f"TorchVision版本: {torchvision.__version__}")

ModuleNotFoundError: No module named 'torch'